In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
import open3d as o3d
import torch
from matplotlib import cm
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import matplotlib.pyplot as plt

from imps.data.scannet import ScanNetScene, CLASS_NAMES
from imps.sqn.model import SQN
from imps.sqn.data_utils import prepare_input
from imps.point_augment.Augmentor.augmentor import Augmentor
import imps.point_augment.Common.data_utils as d_utils

from random import choice
from random import random, sample, uniform


import albumentations as A
import volumentations as V
from pathlib import Path
import yaml
import scipy

from copy import deepcopy
from itertools import product

import tqdm


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
SCENE_DIR1 = '/app/mnt/scans/scene0040_00'
SCENE_DIR2 = '/app/mnt/scans/scene0041_00'
SCENE_DIR3 = '/app/mnt/scans/scene0009_00'
SCENE_DIR4 = '/app/mnt/scans/scene0025_00'

image_augmentations_path = "/app/configs/albumentations_aug.yaml"
volume_augmentations_path = "/app/configs/volumentations_aug.yaml"
color_mean_std = "/app/processed/color_mean_std.yaml"

In [25]:
N_POINTS = int(1.5e5)
# Not important we are not using this here yet. Keep this small for quick data processing
RESOLUTION = 25
SIGMAS = None
LABEL_RATIO = 1
N_LABEL = int(N_POINTS*LABEL_RATIO)
DEVICE = 'cuda'
IGNORED_LABELS = [0]

In [4]:
def get_loss(logits, labels, class_weights):
    class_weights = torch.from_numpy(class_weights).float().to(logits.device)
    logits = logits.reshape(-1, len(class_weights))
    labels = labels.reshape(-1)

    criterion = torch.nn.CrossEntropyLoss(weight=class_weights, reduction='none')
    output_loss = criterion(logits, labels)
    output_loss = output_loss.mean()
    
    return output_loss

In [ ]:
def _load_yaml(filepath):
    with open(filepath) as f:
        file = yaml.safe_load(f)
    return file

In [ ]:
def flip_in_center(coordinates):
    # moving coordinates to center
    coordinates -= coordinates.mean(0)
    aug = V.Compose(
        [
            V.Flip3d(axis=(0, 1, 0), always_apply=True),
            V.Flip3d(axis=(1, 0, 0), always_apply=True),
        ]
    )
    first_crop = coordinates[:, 0] > 0
    first_crop &= coordinates[:, 1] > 0
    # x -y
    second_crop = coordinates[:, 0] > 0
    second_crop &= coordinates[:, 1] < 0
    # -x y
    third_crop = coordinates[:, 0] < 0
    third_crop &= coordinates[:, 1] > 0
    # -x -y
    fourth_crop = coordinates[:, 0] < 0
    fourth_crop &= coordinates[:, 1] < 0

    if first_crop.size > 1:
        coordinates[first_crop] = aug(points=coordinates[first_crop])["points"]
    if second_crop.size > 1:
        minimum = coordinates[second_crop].min(0)
        minimum[2] = 0
        minimum[0] = 0
        coordinates[second_crop] = aug(points=coordinates[second_crop])["points"]
        coordinates[second_crop] += minimum
    if third_crop.size > 1:
        minimum = coordinates[third_crop].min(0)
        minimum[2] = 0
        minimum[1] = 0
        coordinates[third_crop] = aug(points=coordinates[third_crop])["points"]
        coordinates[third_crop] += minimum
    if fourth_crop.size > 1:
        minimum = coordinates[fourth_crop].min(0)
        minimum[2] = 0
        coordinates[fourth_crop] = aug(points=coordinates[fourth_crop])["points"]
        coordinates[fourth_crop] += minimum

    return coordinates

In [ ]:
def elastic_distortion(pointcloud, granularity, magnitude):
    """Apply elastic distortion on sparse coordinate space.

    pointcloud: numpy array of (number of points, at least 3 spatial dims)
    granularity: size of the noise grid (in same scale[m/cm] as the voxel grid)
    magnitude: noise multiplier
  """
    blurx = np.ones((3, 1, 1, 1)).astype("float32") / 3
    blury = np.ones((1, 3, 1, 1)).astype("float32") / 3
    blurz = np.ones((1, 1, 3, 1)).astype("float32") / 3
    coords = pointcloud[:, :3]
    coords_min = coords.min(0)

    # Create Gaussian noise tensor of the size given by granularity.
    noise_dim = ((coords - coords_min).max(0) // granularity).astype(int) + 3
    noise = np.random.randn(*noise_dim, 3).astype(np.float32)

    # Smoothing.
    for _ in range(2):
        noise = scipy.ndimage.filters.convolve(noise, blurx, mode="constant", cval=0)
        noise = scipy.ndimage.filters.convolve(noise, blury, mode="constant", cval=0)
        noise = scipy.ndimage.filters.convolve(noise, blurz, mode="constant", cval=0)

    # Trilinear interpolate noise filters for each spatial dimensions.
    ax = [
        np.linspace(d_min, d_max, d)
        for d_min, d_max, d in zip(
            coords_min - granularity,
            coords_min + granularity * (noise_dim - 2),
            noise_dim,
        )
    ]
    interp = scipy.interpolate.RegularGridInterpolator(
        ax, noise, bounds_error=0, fill_value=0
    )
    pointcloud[:, :3] = coords + interp(coords) * magnitude
    return pointcloud

In [5]:
query_idxs = np.random.choice(N_POINTS, N_LABEL, replace=False)

In [6]:
epochs = list(range(1,51))

In [7]:
scene1 = ScanNetScene(SCENE_DIR1)

voxel_grid1, surface_points1, surface_colors1, vicinity_points1, vicinity_distances1, point_labels1 = scene1.create_if_data(
    RESOLUTION, N_POINTS, SIGMAS
)


In [8]:
#scene 1

class_counts1 = []
for c in range(len(CLASS_NAMES.keys())):
    class_counts1.append(np.sum(point_labels1 == c))
class_counts1 = np.array(class_counts1)

for ign in IGNORED_LABELS:
    class_counts1[ign] = 0
class_weights1 = class_counts1 / class_counts1.sum()


query_points1 = surface_points1[query_idxs]
query_labels1 = point_labels1[query_idxs]
print(query_labels1)
print(query_labels1.shape)

[10  2  0 ... 10  1  0]
(150000,)


In [9]:
self_dim = 3
augmentor = Augmentor().cuda()
optimizer_a = optim.Adam(
            augmentor.parameters(),
            lr=0.001,
            betas=(0.9, 0.999),
            eps=1e-08,
            weight_decay=0.0001
        )


In [10]:
# Single scene

features1 = torch.FloatTensor(surface_colors1).unsqueeze(0).to(DEVICE)
xyz1 = torch.FloatTensor(surface_points1).unsqueeze(0)
query1 = torch.FloatTensor(query_points1).unsqueeze(0).to(DEVICE)
query_labels1 = torch.LongTensor(query_labels1).unsqueeze(0).to(DEVICE)
#input_points1, input_neighbors1, input_pools1 = prepare_input(xyz1, k=16, num_layers=3, sub_sampling_ratio=4, 
                                             #              device=DEVICE)

In [11]:
#sqn1 = SQN(d_feature=3, d_in=8, encoder_dims=[8, 32, 64], len_class= len(CLASS_NAMES), decoder_dims=[64, len(CLASS_NAMES)], device=DEVICE)
optimizer1_sqn = optim.Adam(sqn1.parameters(), lr=1e-3)

NameError: name 'sqn1' is not defined

In [12]:
def aug_loss(pred, pred_aug, query_labels, class_weights):
    ''' Calculate cross entropy loss, apply label smoothing if needed. '''
    mse_fn = torch.nn.MSELoss(reduce=True, size_average=True)
    
    loss = get_loss(pred, query_labels, class_weights)
    loss_aug = get_loss(pred_aug, query_labels, class_weights)
    
    pc_con = F.softmax(pred, dim=-1)#.max(dim=1)[0]
    one_hot = F.one_hot(gold, pred.shape[1]).float()
    pc_con = (pc_con*one_hot).max(dim=1)[0]    
    parameters = torch.max(torch.tensor(NUM).cuda(), torch.exp(pc_con) * NUM).cuda()
    
    # both losses are usable
    aug_diff = 1.0 * torch.abs(1.0 - torch.exp(loss_aug - loss * parameters)).mean()
    #aug_diff =  W*torch.abs(cls_aug_raw - cls_pc_raw*parameters).mean()
    aug_loss = loss_aug + aug_diff

    return aug_loss


In [13]:
def cls_loss(pred, pred_aug, pc_feat, aug_feat):
    ''' Calculate cross entropy loss, apply label smoothing if needed. '''
    mse_fn = torch.nn.MSELoss(reduce=True, size_average=True)

    loss = get_loss(pred, query_labels, class_weights)
    loss_aug = get_loss(pred_aug, query_labels, class_weights)
    
    feat_diff = 10.0*mse_fn(pc_feat,aug_feat)
    parameters = torch.max(torch.tensor(NUM).cuda(), torch.exp(1.0-loss)**2).cuda()
    
    cls_diff = (torch.abs(loss - loss_aug) * (parameters*2)).mean()
    cls_loss = loss + loss_aug  + feat_diff# + cls_diff

    return cls_loss


In [14]:
PointcloudScaleAndTranslate = d_utils.PointcloudScaleAndTranslate() #initiate augmentation
sqn1 = SQN(d_feature=3, d_in=8, encoder_dims=[8, 32, 64], len_class= len(CLASS_NAMES), decoder_dims=[64, len(CLASS_NAMES)], device=DEVICE)
optimizer1_sqn = optim.Adam(sqn1.parameters(), lr=1e-3)
loss_list1=[]

In [27]:
xyz1=xyz1.to(DEVICE)
surface_points_augmented = PointcloudScaleAndTranslate(xyz1).to(DEVICE)
cuda_check = surface_points_augmented.is_cuda
if cuda_check:
    get_cuda_device = surface_points_augmented.get_device()
    
print(get_cuda_device)

tensor([[-0.0626, -0.0578, -0.0221],
        [-0.4710, -0.0841,  0.0873],
        [ 0.2765,  0.2337, -0.1998],
        ...,
        [-0.2421,  0.4181,  0.1388],
        [-0.1444,  0.3924,  0.1471],
        [-0.3464,  0.3534, -0.0728]])


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [21]:
print(xyz1)

tensor([[[-0.0626, -0.0578, -0.0221],
         [-0.4710, -0.0841,  0.0873],
         [ 0.2765,  0.2337, -0.1998],
         ...,
         [-0.2421,  0.4181,  0.1388],
         [-0.1444,  0.3924,  0.1471],
         [-0.3464,  0.3534, -0.0728]]])


In [26]:
xyz1=xyz1.to(DEVICE)

In [27]:


for e in range(50):
    
    surface_points_augmented = PointcloudScaleAndTranslate(xyz1) # I do the augmentation
    surface_points_augmented = surface_points_augmented.transpose(2, 1).contiguous()
    
    noise = 0.02 * torch.randn(1, 1024).cuda() #adding noise for creation of M and D matrix in the augmentator module
    
    sqn1 = sqn1.train()
    augmentor = augmentor.train()
    
    optimizer_a.zero_grad()
    optimizer1_sqn.zero_grad()
    
    aug_pc = augmentor(surface_points_augmented, noise) # obtaining the augmented point cloud
    # SQN without augmentation
    input_points1, input_neighbors1, input_pools1 = prepare_input(surface_points_augmented, k=16, num_layers=3, sub_sampling_ratio=4, 
                                                           device=DEVICE)
    logits1, lat_feat1, query_feat1  = sqn1.forward(features1, input_points1, input_neighbors1, input_pools1, query1)
    
    #SQN with augmentation
    input_points1_aug, input_neighbors1_aug, input_pools1_aug = prepare_input(surface_points_augmented, k=16, num_layers=3, sub_sampling_ratio=4, 
                                                           device=DEVICE)
    logits1_aug, lat_feat_aug, query_feat_aug = sqn1.forward(features1, input_points1_aug, input_neighbors1_aug, input_pools1_aug, query1)
    
    augLoss  = aug_loss(logits1, logits1_aug, query_labels1, class_weights1)

    clsLoss = cls_loss(logits1, logits1_aug, lat_feat1,
                                lat_feat_aug)


    augLoss.backward(retain_graph=True)
    clsLoss.backward(retain_graph=True)

    optimizer_c.step()
    optimizer1_sqn.step()
    
    train_acc = eval_one_epoch(sqn1.eval(), trainDataLoader)
    #test_acc = eval_one_epoch(sqn1.eval(), testDataLoader)

    print('CLS Loss: {}'.format(round(clsLoss.data)))
    print('AUG Loss: %.2f'%augLoss.data)

    print('Train Accuracy: %f' % train_acc)
    #self.log_string('Test Accuracy: %f'%test_acc)

    #writer.add_scalar("Train_Acc", train_acc, epoch)
   # writer.add_scalar("Test_Acc", test_acc, epoch)
    
    print(f"Epoch {e+1}: {round(loss1.item(), 4)}")
    loss_list1.append(round(loss1.item(), 4))
    
plt.plot(epochs, loss_list1)
plt.show()

tensor([[ 0.2951,  0.2427, -0.0070],
        [-0.3027,  0.3994, -0.0803],
        [ 0.1859,  0.2980, -0.0850],
        ...,
        [-0.1559,  0.3550,  0.0449],
        [ 0.1637, -0.3562, -0.1952],
        [-0.3131,  0.3978,  0.1608]], device='cuda:0')
0


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 512])